# **NLP2_Run3**

In [1]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 69.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.1 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [2]:

import openai
import os
from IPython.display import Markdown


### **Accessing the GPT4 API**

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']
openai.api_key = my_api_key


### **Generate Mathematical Model**

In [4]:
problem = """You are a chemist in charge of a process that requires two liquids to reach the end product. Your goal is to maximize the quality of the product in the end, which is measured by the difference between the Napierian logarithm of the product of the litres used of each component and the cube root of the summation of the first component litres as well as double the second component litres. It is required that the squared difference between the litres employed of each component and the mean litres utilised of both components is below ten. This is necessary for the obtained set to be stable. Furthermore, the mixture tank available to you can only fit 80 l. Please formulate a mathematical optimization model for this problem. """

In [5]:

client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the variables for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 3
)


In [6]:
print(response1.choices[0].message.content)

Variables:
1. X1: Litres of liquid 1 used in the process.
2. X2: Litres of liquid 2 used in the process.


In [7]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the objective function for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 3
)

In [8]:
print(response2.choices[0].message.content)

The objective function for the problem is: 

Maximize F(X1, X2) = ln(X1*X2) - (X1 + 2*X2)^(1/3)


In [9]:
response3 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the constraints for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content +  response2.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 3
)

In [10]:
print(response3.choices[0].message.content)

The constraints for this mathematical optimization problem are:

Constraint 1: (X1 - (X1+X2)/2)^2 + (X2 - (X1 + X2)/2)^2 ≤ 10

Constraint 2: X1 + X2 ≤ 80

Additional non-negativity constraints: 
X1 ≥ 0
X2 ≥ 0


### **Generate Pyomo Code**

In [11]:
response4 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content + response2.choices[0].message.content + response3.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 3
)

In [12]:
Markdown(response4.choices[0].message.content)

Sure, here is a Python code using Pyomo for the described optimization problem. A key point is the second-order cone constraint, which is usually nontrivial for solver support in Python optimization libraries. I will use 'ipopt' to solve the problem, which is a solver that can handle this type of constraints.

Please install pyomo and ipopt if they are not available in your python environment.

```python
from __future__ import division
from pyomo.environ import *
from pyomo.opt import SolverFactory
from math import log

# Create a simple model
model = ConcreteModel()

# Ipopt treats everything as a minimization, so we flip the sign on the objective to achieve maximization
def obj_expression(model): 
    return -(log(model.x1*model.x2) - (model.x1 + 2*model.x2)**(1/3))

model.OBJ = Objective(rule=obj_expression)

# Variables
model.x1 = Var(within=NonNegativeReals)
model.x2 = Var(within=NonNegativeReals)

# Constraints
def con1_rule(model):
    return ((model.x1 - (model.x1+model.x2)/2)**2 + (model.x2-(model.x1 + model.x2)/2)**2)<=10 
model.AddConst1 = Constraint(rule=con1_rule)

def con2_rule(model): 
    return model.x1 + model.x2 <=80
model.AddConst2 = Constraint(rule=con2_rule)

# Optimize
opt = SolverFactory('ipopt')
opt.solve(model)

# Print the value of the variables at the optimum
print("**** Results ****")
print("X1 = ", value(model.x1))
print("X2 = ", value(model.x2))
```
This program is a concrete Pyomo model, which means that the data (in this case the coefficients in the objective and constraint functions) are directly included in the formulation.

### **Run the code Generated by GPT4**

In [13]:
# Install Python API for AMPL
!pip install amplpy --upgrade

from amplpy import AMPL,ampl_notebook

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 33.7 MB/s eta 0:00:00


In [14]:
_ = load_dotenv(find_dotenv('second_knitro_license.env'))
my_knitro_license = os.environ['knitro_license']

ampl = ampl_notebook(
    modules=["knitro"],  # modules to install
    license_uuid=my_knitro_license,  # license to use
)

Licensed to AMPL Community Edition License for <cevikmelis14@gmail.com>.


In [15]:
from pyomo.environ import *
from pyomo.opt import SolverFactory
from math import log

# Create a simple model
model = ConcreteModel()

# Ipopt treats everything as a minimization, so we flip the sign on the objective to achieve maximization
def obj_expression(model):
    return -(log(model.x1*model.x2) - (model.x1 + 2*model.x2)**(1/3))

model.OBJ = Objective(rule=obj_expression)

# Variables
model.x1 = Var(within=NonNegativeReals)
model.x2 = Var(within=NonNegativeReals)

# Constraints
def con1_rule(model):
    return ((model.x1 - (model.x1+model.x2)/2)**2 + (model.x2-(model.x1 + model.x2)/2)**2)<=10
model.AddConst1 = Constraint(rule=con1_rule)

def con2_rule(model):
    return model.x1 + model.x2 <=80
model.AddConst2 = Constraint(rule=con2_rule)

# Optimize
opt = SolverFactory('knitro')
opt.solve(model)

# Print the value of the variables at the optimum
print("**** Results ****")
print("X1 = ", value(model.x1))
print("X2 = ", value(model.x2))

ERROR:pyomo.core:Rule failed when generating expression for Objective OBJ with index None:
AttributeError: 'ConcreteModel' object has no attribute 'x1'
ERROR:pyomo.core:Constructing component 'OBJ' from data=None failed:
    AttributeError: 'ConcreteModel' object has no attribute 'x1'


AttributeError: 'ConcreteModel' object has no attribute 'x1'

In [23]:
from pyomo.environ import *
from pyomo.opt import SolverFactory
from math import log

# Create a simple model
model = ConcreteModel()

model.x1 = Var(within=NonNegativeReals)
model.x2 = Var(within=NonNegativeReals)


# Ipopt treats everything as a minimization, so we flip the sign on the objective to achieve maximization
def obj_expression(model):
    return -(log(model.x1*model.x2) - (model.x1 + 2*model.x2)**(1/3))

model.OBJ = Objective(rule=obj_expression)

# Constraints
def con1_rule(model):
    return ((model.x1 - (model.x1+model.x2)/2)**2 + (model.x2-(model.x1 + model.x2)/2)**2)<=10
model.AddConst1 = Constraint(rule=con1_rule)

def con2_rule(model):
    return model.x1 + model.x2 <=80
model.AddConst2 = Constraint(rule=con2_rule)

# Optimize
opt = SolverFactory('ipopt')
opt.solve(model)

# Print the value of the variables at the optimum
print("**** Results ****")
print("X1 = ", value(model.x1))
print("X2 = ", value(model.x2))

ERROR:pyomo.core:Rule failed when generating expression for Objective OBJ with index None:
TypeError: Implicit conversion of Pyomo numeric value (x1*x2) to float is disabled.
This error is often the result of using Pyomo components as arguments to
one of the Python built-in math module functions when defining
expressions. Avoid this error by using Pyomo-provided math functions or
explicitly resolving the numeric value using the Pyomo value() function.
ERROR:pyomo.core:Constructing component 'OBJ' from data=None failed:
    TypeError: Implicit conversion of Pyomo numeric value (x1*x2) to float is disabled.
This error is often the result of using Pyomo components as arguments to
one of the Python built-in math module functions when defining
expressions. Avoid this error by using Pyomo-provided math functions or
explicitly resolving the numeric value using the Pyomo value() function.


TypeError: Implicit conversion of Pyomo numeric value (x1*x2) to float is disabled.
This error is often the result of using Pyomo components as arguments to
one of the Python built-in math module functions when defining
expressions. Avoid this error by using Pyomo-provided math functions or
explicitly resolving the numeric value using the Pyomo value() function.

### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**

In [24]:
from pyomo.environ import *
from pyomo.opt import SolverFactory


# Create a simple model
model = ConcreteModel()

# Variables
model.x1 = Var(within=NonNegativeReals)
model.x2 = Var(within=NonNegativeReals)

# Ipopt treats everything as a minimization, so we flip the sign on the objective to achieve maximization
def obj_expression(model):
    return log(model.x1*model.x2) - (model.x1 + 2*model.x2)**(1/3)

model.OBJ = Objective(rule=obj_expression, sense=maximize)

# Constraints
def con1_rule(model):
    return ((model.x1 - (model.x1+model.x2)/2)**2 + (model.x2-(model.x1 + model.x2)/2)**2)<=10
model.AddConst1 = Constraint(rule=con1_rule)

def con2_rule(model):
    return model.x1 + model.x2 <=80
model.AddConst2 = Constraint(rule=con2_rule)

# Optimize
opt = SolverFactory('knitro')
opt.solve(model)

# Print the value of the variables at the optimum
print("**** Results ****")
print("X1 = ", value(model.x1))
print("X2 = ", value(model.x2))

**** Results ****
X1 =  42.23606797533548
X2 =  37.76393202225084
